In [36]:
import triangle as tr
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
import os
from scipy.io import savemat

def show_mesh(points, triangles):
    # Normalisation pour l'aspect ratio
    x_, y_, z_ = points
    fig_scale = max(max(x_) - min(x_), max(y_) - min(y_))
    x_scale = (max(x_) - min(x_)) / fig_scale
    y_scale = (max(y_) - min(y_)) / fig_scale
    z_scale = 0.3  # Z scale arbitraire pour une meilleure visualisation

    # Création de la figure 3D avec Plotly
    fig = ff.create_trisurf(
        x=x_,
        y=y_,
        z=z_,
        simplices=triangles[:3, :].T,
        color_func=list(range(len(triangles[:3, :].T))),  # Couleurs basées sur les indices des triangles
        show_colorbar=True,
        title="Maillage triangulaire",
        aspectratio=dict(x=x_scale, y=y_scale, z=z_scale)
    )
    # Afficher la figure
    fig.show()

In [37]:
def show_mesh(points, triangles, feed_point, color="red"):
    # Normalisation pour l'aspect ratio
    x_, y_, z_ = points
    fig_scale = max(max(x_) - min(x_), max(y_) - min(y_))
    x_scale = (max(x_) - min(x_)) / fig_scale
    y_scale = (max(y_) - min(y_)) / fig_scale
    z_scale = 0.3  # Z scale arbitraire pour une meilleure visualisation

    # Création de la figure 3D avec Plotly
    fig = ff.create_trisurf(
        x=x_,
        y=y_,
        z=z_,
        simplices=triangles[:3, :].T,
        color_func=list(range(len(triangles[:3, :].T))),  # Couleurs basées sur les indices des triangles
        show_colorbar=False,
        title="Maillage triangulaire",
        aspectratio=dict(x=x_scale, y=y_scale, z=z_scale)
    )

    # Ajouter le point spécifique
    scatter = go.Scatter3d(
        x=[feed_point[0]],
        y=[feed_point[1]],
        z=[feed_point[2]],
        mode="markers",
        marker=dict(size=6, color=color, opacity=1.0),
        name="Point d'alimentation"
    )

    # Ajouter le point à la figure
    fig.add_trace(scatter)

    # Afficher la figure
    fig.show()

In [38]:
# Fonction pour générer les points de l'antenne IFA
def generate_ifa_antenna_points(L, H, ML, MH, gap, Lp):
    points = np.array([
        [0, 0], [L, 0], [L, H], [0, H], [0, H + MH], [L, H + MH], [L, H + gap], 
        [L - ML, H + gap], [L - ML, H + MH - ML], [2 * ML + Lp, H + MH - ML], 
        [2 * ML + Lp, H], [ML + Lp, H], [ML + Lp, H + MH - ML], [ML, H + MH - ML], [ML, H]
    ])
    return points

In [39]:
# Fonction pour générer les segments de l'antenne IFA
def generate_ifa_antenna_segments():
    segments = np.array([
        [0, 1], [1, 2], [2, 10], [10, 9], [9, 8], [8, 7], [7, 6], [6, 5], [5, 4], [4, 3], 
        [0, 3], [11, 12], [12, 13], [13, 14], [11, 14]
    ])
    return segments

In [40]:
def data_save(filename, save_folder_name, points, triangle):
    data = {'p': points, 't': triangle}
    save_file_name = filename + '.mat'
    full_save_path = os.path.join(save_folder_name, save_file_name)
    if not os.path.exists(save_folder_name):
        os.makedirs(save_folder_name)
        print(f"Directory '{save_folder_name}' created.")
    savemat(full_save_path, data)
    print(f"Data saved successfully to {full_save_path}")
    return save_file_name

In [41]:
# Définition des dimensions de l'antenne IFA
L = 0.04
H = 0.035
ML = 0.004
MH = 0.015
gap = 0.0015
Lp = 0.01

# Génération des points et des segments
vertices = generate_ifa_antenna_points(L, H, ML, MH, gap, Lp)
segments = generate_ifa_antenna_segments()

In [42]:
# Définition des trous (un point à l'intérieur de la zone à exclure)
holes = np.array([
    [(ML + ML + Lp) / 2, (H + (H + MH - ML)) / 2]
])

# Préparation des données pour la bibliothèque Triangle
ifa_antenna = {
    'vertices': vertices,
    'segments': segments,
    'holes': holes
}

In [43]:
mesh = tr.triangulate(ifa_antenna, 'pq30a0.000005Dj')

nbr_of_points = mesh['vertices'].shape[0]
nbr_of_triangles = mesh['triangles'].shape[0]
print(f"Nombre de points = {nbr_of_points}")
print(f"Nombre de triangles = {nbr_of_triangles}")

triangles = np.zeros((4, nbr_of_triangles), dtype=int)
triangles[:3, :] = mesh['triangles'].T
points = np.zeros((3, nbr_of_points))
points[:2, :] = mesh['vertices'].T

print(f"Matrice points shape = {points.shape}")
print(f"Matrice triangles shape = {triangles.shape}")

feed_point_IFA = [0.016, 0.046, 0]
# feed_point_IFA = [0.004, 0.035, 0]
# feed_point_IFA = [0.016, 0.035, 0]
show_mesh(points, triangles, feed_point_IFA, color="red")  # Change "black" si besoin


Nombre de points = 329
Nombre de triangles = 543
Matrice points shape = (3, 329)
Matrice triangles shape = (4, 543)


In [44]:
data_save("IFA2", "data/antennas_mesh/", points, triangles)

Data saved successfully to data/antennas_mesh/IFA2.mat


'IFA2.mat'